Below are parameters for the Papercraft verifiable delay function (VDF).

In [1]:
from estimator.rok_estimator import *

In [2]:
mode = "C.L"

secpar = 128
kappa = 80
log_q = 62
rep = 57
t = 64 * 22
T = t * rep
n_vdf = 14 # Candidate module rank for VDF. Try bigger ones if the code complains. It should only depend on the conductor f of the cyclotomic ring and log_q. 

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 44,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel" : 2,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "base": 21,
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = [
    ("bdecomp", {"base": ops_params["base"]}), 
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

# ops = 9 * loop + [("finish", {})]
ops = small_loop + big_loop + small_loop + big_loop + small_loop +  big_loop + [("finish", {})]

sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

print(f'VDF rounds T = {T}')


Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1222144 |  57 |        ( 14 | 14 )        |         (  0 | 14 )         | 66.44 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  1222144 |  72 |        ( 14 | 14 )        |         (  0 | 14 )         | 83.92 MB |      (153.8 KB | 153.8 KB)      |         (2^-120 | 2^-120)         
 split     |   611072 | 144 |        ( 14 | 32 )        |         (  0 | 29 )         | 83.92 MB |      (732.4 KB | 886.2 KB)      |         (2^-124 | 2^-120)         
 fold      |   611072 |  57 |        ( 22 | 26 )        |         (  9 | 26 )         | 332.2 MB |      (732.4 KB | 886.2 KB)      |         (2^-83  | 2^-83 )         
 bdecomp   |   611072 | 171 |        ( 17 | 17 )        |         (  4 | 17 )         | 498.3 MB |      (1.065 MB | 1.930 MB)      |         (2